In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

from upcycle.plotting.credible_regions import get_gaussian_region

sns.set(font_scale=1.5, style='whitegrid')

In [2]:
def get_arm(exp_dir, arm_name, table_name, x_col, y_col, window=1):
    arm_path = Path(exp_dir) / arm_name
    arm_dfs = [pd.read_csv(f) for f in arm_path.rglob(f'*{table_name}*')]
    print(f"{len(arm_dfs)} tables found in {arm_path.as_posix()}")
    arm_data = np.stack([df[y_col].rolling(window, min_periods=1).mean() for df in arm_dfs])
    x_range = arm_dfs[0][x_col].values
    mean, lb, ub = get_gaussian_region(arm_data.mean(0), arm_data.var(0))
    return x_range, mean, lb, ub

In [6]:
palette = sns.color_palette('bright')

exp_dir = '../data/experiments/regression/num_inducing_ablation'
dataset = 'elevators'
model_types = ['wiski_gp_regression', 'svgp_regression']
num_inducing = [64, 144, 256]
model_labels = dict(exact_gp_regression='Exact', wiski_gp_regression='WISKI', svgp_regression='O-SVGP')
model_colors = dict(exact_gp_regression=palette[7], wiski_gp_regression=palette[4], svgp_regression=palette[0])
model_versions = dict(exact_gp_regression='v0.0.10', wiski_gp_regression='v0.0.11', svgp_regression='v0.0.11')
window = 4

In [7]:
model_type = 'svgp_regression'
fig = plt.figure(figsize=(5.5, 5))
for m in num_inducing:
    arm_name = f"{model_type}-{m}-{dataset}-v0.0.10"
    x_range, mean, lb, ub = get_arm(exp_dir, arm_name, 'online_metrics', 'step', 'test_nll', window)
    
    label = f'$m={m}$'
    plt.plot(x_range, mean, label=label)
    plt.fill_between(x_range, lb, ub, alpha=0.25)

plt.xlabel('t')
plt.ylabel('NLL')
plt.legend()
plt.ylim((1, 3))
plt.tight_layout()
# plt.savefig(f"figures/regression/num_inducing_ablation_{model_type}_{dataset}.pdf")

0 tables found in ../data/experiments/regression/num_inducing_ablation/svgp_regression-64-elevators-v0.0.10


ValueError: need at least one array to stack

<Figure size 396x360 with 0 Axes>

In [8]:
model_type = 'wiski_gp_regression'
fig = plt.figure(figsize=(5.5, 5))
for m in num_inducing:
    arm_name = f"{model_type}-{m}-{dataset}-v0.0.11"
    x_range, mean, lb, ub = get_arm(exp_dir, arm_name, 'online_metrics', 'step', 'test_nll', window)
    
    plt.plot(x_range, mean, label=m)
    plt.fill_between(x_range, lb, ub, alpha=0.25)

plt.xlabel('t')
plt.ylabel('NLL')
plt.legend()
plt.ylim((1, 3))
plt.tight_layout()
# plt.savefig(f"figures/regression/num_inducing_ablation_{model_type}_{dataset}.pdf")

0 tables found in ../data/experiments/regression/num_inducing_ablation/wiski_gp_regression-64-elevators-v0.0.11


ValueError: need at least one array to stack

<Figure size 396x360 with 0 Axes>